In [ ]:
report_url = input('PrimeXBT Report URL')
trade_group = input('Trade Group')

In [ ]:
import pandas as pd

html = pd.read_html(report_url)

In [ ]:
import numpy as np

df_prime = html[-1]
df_prime.columns = df_prime.columns\
    .str.strip()\
    .str.replace(r'[\W]', '_')\
    .str.replace(r'___UTC_', '')

df_prime = df_prime.replace('—', np.nan)
df_prime['Exit_Transfer_Time'] = pd.to_datetime(df_prime.Exit_Transfer_Time, format='%d/%m/%Y %H:%M')

In [ ]:
df_cointracking = pd.DataFrame(columns=[
    'Type', 'Buy Amount', 'Buy Cur.', 'Sell Amount', 'Sell Cur.',
    'Fee Amount', 'Fee Cur.', 'Exchange', 'Trade Group', 'Comment',
    'Date', 'Liquidity Pool'])

for index, row in df_prime.iterrows():
    entry = dict(
        Exchange='PrimeXBT Margin',
        Date=row.Exit_Transfer_Time,
        Comment=row.Comment)

    entry_wallet = dict(
        Exchange='PrimeXBT Wallet',
        Date=row.Exit_Transfer_Time,
        Comment=row.Comment)

    if row.Entry_time in ['TRADE COMMISSION', 'FINANCING']:
        entry['Type'] = 'Margin Fee' if row.Entry_time == 'TRADE COMMISSION' else 'Borrowing Fee'
        entry['Sell Amount'] = abs(row.Profit_Loss)
        entry['Sell Cur.'] = 'USDT'

    elif row.Entry_time == 'DEPOSIT':
        entry['Type'] = 'Deposit'
        entry['Buy Amount'] = row.Profit_Loss
        entry['Buy Cur.'] = 'USDT'

        entry_wallet['Type'] = 'Withdrawal'
        entry_wallet['Sell Amount'] = abs(row.Profit_Loss)
        entry_wallet['Sell Cur.'] = 'USDT'

    elif row.Entry_time == 'WITHDRAWAL':
        entry['Type'] = 'Withdrawal'
        entry['Sell Amount'] = abs(row.Profit_Loss)
        entry['Sell Cur.'] = 'USDT'

        entry_wallet['Type'] = 'Deposit'
        entry_wallet['Buy Amount'] = abs(row.Profit_Loss)
        entry_wallet['Buy Cur.'] = 'USDT'

    elif row.Profit_Loss > 0:
        entry['Type'] = 'Derivatives / Futures Profit'
        entry['Buy Amount'] = abs(row.Profit_Loss)
        entry['Buy Cur.'] = 'USDT'

    elif row.Profit_Loss < 0:
        entry['Type'] = 'Derivatives / Futures Loss'
        entry['Sell Amount'] = abs(row.Profit_Loss)
        entry['Sell Cur.'] = 'USDT'

    df_cointracking = pd.concat([
        df_cointracking,
        pd.DataFrame.from_records([entry])])

    if row.Entry_time in ['WITHDRAWAL', 'DEPOSIT']:
        df_cointracking = pd.concat([
            df_cointracking,
            pd.DataFrame.from_records([entry_wallet])])

df_cointracking = df_cointracking.fillna('')
df_cointracking['Date'] = df_cointracking.Date.dt.strftime('%d-%m-%Y %H:%M:%S')
df_cointracking.to_excel('CoinTracking_Excel_Import.xls', index=False)
df_cointracking